In [1]:
import numpy as np
import pandas as pd # for loading data

In [2]:
cd ..

/home/nicktehrany/Documents/Uni/3rd Year/P4/Machine Learning/Project


In [3]:
# Importing our own Functions
from util.helperfunctions import one_hot, one_hot_genres, cnn_clean_text, one_hot_reverse

/home/nicktehrany/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/nicktehrany/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/nicktehrany/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/nicktehrany/.local/lib/python3.6/site-packages/tensorflow/python/frame

In [4]:
cd src

/home/nicktehrany/Documents/Uni/3rd Year/P4/Machine Learning/Project/src


In [5]:
train_df = pd.read_csv("train.csv")

In [6]:
lyrics, max_len, word_list = cnn_clean_text(train_df['Lyrics'], 1)

# writing cleaned text back to df
index = 0
for text in lyrics:
    train_df['Lyrics'][index] = text
    index+=1
del lyrics

In [7]:
words = []
words.append('<PAD>')
for word in word_list:
    words.append(word)

In [8]:
train_lyrics = np.zeros(shape=(len(train_df), max_len))

row_index = 0
for row in train_df['Lyrics']:
    single_words = row.split()
    col_index = 0
    for word in single_words:
        train_lyrics[row_index][col_index] = words.index(word)
        col_index+=1
    row_index+=1

In [9]:
print(train_lyrics.shape)

(10098, 1525)


In [14]:
train_genres = one_hot_genres(train_df['Genre'])

In [15]:
val_df = pd.read_csv("validation.csv")
lyrics, dummy_len, dummy_list = cnn_clean_text(val_df['Lyrics'], 0)

# writing cleaned text back to df
index = 0
for text in lyrics:
    val_df['Lyrics'][index] = text
    index+=1
del lyrics

In [16]:
val_genres = one_hot_genres(val_df['Genre'])

In [17]:
val_lyrics = np.zeros(shape=(len(val_df), max_len))

row_index = 0
for row in val_df['Lyrics']:
    single_words = row.split()
    col_index = 0
    for word in single_words:
        if word in word_list:
            val_lyrics[row_index][col_index] = words.index(word)
        col_index+=1
    row_index+=1

In [18]:
test_df = pd.read_csv("test.csv")
lyrics, dummy_len, dummy_list = cnn_clean_text(test_df['Lyrics'], 0)

# writing cleaned text back to df
index = 0
for text in lyrics:
    test_df['Lyrics'][index] = text
    index+=1
del lyrics

In [19]:
test_genres = one_hot_genres(test_df['Genre'])

In [20]:
test_lyrics = np.zeros(shape=(len(test_df), max_len))

row_index = 0
for row in test_df['Lyrics']:
    single_words = row.split()
    col_index = 0
    for word in single_words:
        if word in word_list:
            test_lyrics[row_index][col_index] = words.index(word)
        col_index+=1
    row_index+=1

In [ ]:
word2vec = {}
with open('w2v_cbow_model.word2vec', encoding='UTF-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vec = np.asarray(values[1:], dtype='float32')
        word2vec[word] = vec

In [ ]:
EMBEDDING_DIM = 100
embedding_matrix = np.zeros((len(words), EMBEDDING_DIM))
index = 0
for word in words:
    embedding_vector = word2vec.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[index] = embedding_vector
    index+=1


In [ ]:
from keras.layers import Embedding

embedding_layer = Embedding(len(words),
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=max_len,
                            trainable=False)

In [85]:
from keras.models import Model
from keras.layers import Dense, Input, Activation, Conv1D, MaxPooling1D, Dropout, Flatten

sequence_input = Input(shape=(max_len,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
l_cov1= Conv1D(128, 5, activation='relu')(embedded_sequences)
l_pool1 = MaxPooling1D(5)(l_cov1)
l_cov2 = Conv1D(12, 5, activation='relu')(l_pool1)
l_pool2 = MaxPooling1D(5)(l_cov2)
l_cov3 = Conv1D(12, 5, activation='relu')(l_pool2)
l_pool3 = MaxPooling1D(35)(l_cov3)  # global max pooling
l_flat = Flatten()(l_pool3)
l_dense = Dense(12, activation='relu')(l_flat)
preds = Dense(8, activation='softmax')(l_dense)
model = Model(sequence_input, preds)

model.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        (None, 1525)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 1525, 100)         3426100   
_________________________________________________________________
conv1d_16 (Conv1D)           (None, 1521, 128)         64128     
_________________________________________________________________
max_pooling1d_16 (MaxPooling (None, 304, 128)          0         
_________________________________________________________________
conv1d_17 (Conv1D)           (None, 300, 12)           7692      
_________________________________________________________________
max_pooling1d_17 (MaxPooling (None, 60, 12)            0         
_________________________________________________________________
conv1d_18 (Conv1D)           (None, 56, 12)            732 

In [86]:
from keras.optimizers import Adam

optimizer = Adam(learning_rate=0.0001)
model.compile(loss='categorical_crossentropy',
              optimizer='Adam',
              metrics=['acc'])

model.fit(train_lyrics, train_genres, epochs=5, shuffle=True, batch_size=16, validation_data=(val_lyrics, val_genres))

Train on 10098 samples, validate on 2142 samples
Epoch 1/5
  832/10098 [=>............................] - ETA: 28s - loss: 2.0463 - acc: 0.1743

KeyboardInterrupt: 

In [77]:
results = model.evaluate(test_lyrics, test_genres, batch_size=16)
print("Accuracy: ", results[1])

2160/2160 [==============================] - 2s 975us/step
Accuracy:  0.36250001192092896


In [84]:
from sklearn.metrics import confusion_matrix

y_pred=model.predict_classes(cnn_lyrics_test)
cm=confusion_matrix(one_hot_reverse(test_genres),y_pred)
print(cm)

AttributeError: 'Model' object has no attribute 'predict_classes'

In [ ]:
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

genre_list = ['Pop','Rock', 'Hip-Hop/Rap', 'Country', 'R&B/Soul', 'Metal', 'Alternative/Indie', 'Folk']

df_cm = pd.DataFrame(cm, index = genre_list, columns = genre_list)
print(df_cm)
plt.figure(figsize = (10,7))
sn.heatmap(df_cm, annot=True, linewidth=0.5, fmt='g', cmap="BuPu")
b, t = plt.ylim() # discover the values for bottom and top
b += 0.5 # Add 0.5 to the bottom
t -= 0.5 # Subtract 0.5 from the top
plt.ylim(b, t) # update the ylim(bottom, top) values